<a href="https://colab.research.google.com/github/daniyal9538/CID/blob/master/modify_resolution_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
from google.colab import files
import cv2
from matplotlib import pyplot as plt

In [0]:
from google.colab import auth
auth.authenticate_user()

# https://cloud.google.com/resource-manager/docs/creating-managing-projects
project_id = 'test-demo-222423'
bucket_name = 'cid_training_data'
# Create the service client.
from googleapiclient.discovery import build
gcs_service = build('storage', 'v1')

from apiclient.http import MediaIoBaseDownload

In [0]:
output_width=160
output_height=120
start_file = 39
end_file = 41

In [0]:
for i in range(start_file, end_file+1):
  try:
    download_name = "training_data-{}.npy".format(i)
    path_name = "cid_data-p-t/training_data-12-p-t-{}.npy".format(i)
    new_name = "mod_data-160-120-{}.npy".format(i)
    
    with open(download_name, 'wb') as f:
      # Download the file from a given Google Cloud Storage bucket.
      request = gcs_service.objects().get_media(bucket=bucket_name,
                                                    object=path_name)
      media = MediaIoBaseDownload(f, request)

      done = False
      while not done:
            # _ is a placeholder for a progress object that we ignore.
            # (Our file is small, so we skip reporting progress.)
        _, done = media.next_chunk()  
      print(download_name, 'download complete')
      
  except Exception as ex:
    print("Download failed file", download_name, ex)
  
  try:
    old_data = np.load(download_name)
    new_data = []
  
    for d in old_data:
      screen = d[0]
      screen = cv2.resize(screen, (output_width,output_height))
      # run a color convert:
      screen = cv2.cvtColor(screen, cv2.COLOR_RGB2GRAY)
      new_data.append([screen, d[1]])
    np.save(new_name, new_data)
    print(new_name, "successfully populated")
    
  except Exception as ex:
    print("New failed file", new_name, ex)
  
  
  try:
    !gsutil cp $new_name gs://{bucket_name}/mod_cid_data
    print(new_name, "successfully uploaded to bucket")
  
  except Exception as ex:
    print("Upload failed", new_name, ex)
    
  try:
    !rm -rf $download_name
    !rm -rf $new_name
    new_data.clear()
    
  except Exception as ex:
    print("deletion failed", ex)
     

training_data-39.npy download complete
mod_data-160-120-39.npy successfully populated
Copying file://mod_data-160-120-39.npy [Content-Type=application/octet-stream]...
==> NOTE: You are uploading one or more large file(s), which would run
significantly faster if you enable parallel composite uploads. This
feature can be enabled by editing the
"parallel_composite_upload_threshold" value in your .boto
configuration file. However, note that if you do this large files will
be uploaded as `composite objects
<https://cloud.google.com/storage/docs/composite-objects>`_,which
means that any user who downloads such objects will need to have a
compiled crcmod installed (see "gsutil help crcmod"). This is because
without a compiled crcmod, computing checksums on composite objects is
so slow that gsutil disables downloads of composite objects.

-
Operation completed over 1 objects/278.1 MiB.                                    
mod_data-160-120-39.npy successfully uploaded to bucket
training_data-40